# Recopilación de datos 

Pipeline de recopilación de datos para identificar las variables socioeconómicas que afectan el crecimiento en la generación y publicación de datos sobre biodiversidad en el nodo nacional SiB Colombia, a partir del análisis de datos históricos y de acceso público.

Este pipeline realiza una descarga de datos de fuente como el Banco Mundial a partir de su API y de GBIF a partir de un web scrapping sobre los archivos compilados y procesados por el mismo GBIF sobre las tendenciales globales, regionales y por país.

Adicionalmente, se agregan otras fuentes como la vinculación de los países a la OPG y se explora la plataforma ODIN.

In [3]:
#%pip install wbgapi
import wbgapi as wb
import pandas as pd
import requests
from io import StringIO
from datetime import datetime
import math
import openpyxl


import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller


# Estilo para los gráficos
plt.style.use('seaborn-whitegrid')
plt.rcParams['font.family'] = 'DejaVu Sans'

import missingno as msno
import scipy.stats as stats
from matplotlib.patches import Patch
from scipy.stats import chi2_contingency, chi2
import ruptures as rpt

from sklearn.preprocessing import OneHotEncoder


import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

# Métricas
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 📊 Extracción de datos del Banco Mundial usando el API

En este paso se realiza una selección de **15 variables socioeconómicas**, agrupadas por temática:

## Ciencia y tecnología

- `IP.JRN.ARTC.SC`: Artículos en publicaciones científicas y técnicas  
- `SP.POP.SCIE.RD.P6`: Investigadores dedicados a investigación y desarrollo (por cada millón de personas)  
- `GB.XPD.RSDV.GD.ZS`: Gasto en investigación y desarrollo (% del PIB)  
- `SP.POP.TECH.RD.P6`: Técnicos de investigación y desarrollo (por cada millón de personas)  


## Economía y crecimiento

- `NY.GDP.PCAP.CD`: PIB per cápita (US$ a precios actuales)  

## Educación

- `SE.XPD.TOTL.GD.ZS`: Gasto público en educación, total (% del PIB)  
- `SE.PRM.ENRR`: Inscripción escolar, nivel primario (% bruto)  
- `SE.SEC.ENRR`: Inscripción escolar, nivel secundario (% bruto)  
- `SE.TER.ENRR`: Inscripción escolar, nivel terciario (% bruto)  
- `SE.XPD.TOTL.GB.ZS`: Gasto público en educación, total (% del gasto del gobierno)  

## Cambio climático

- `ER.PTD.TOTL.ZS`: Áreas protegidas terrestres y marinas (% del total de la superficie territorial)  
- `AG.LND.FRST.K2`: Área selvática (kilómetros cuadrados)  


## Ambiente

- `AG.SRF.TOTL.K2`: Superficie (kilómetros cuadrados)  


## Sector público

- `GE.EST`: Government Effectiveness (Estimate)  


## Infraestructura

- `IT.NET.USER.ZS`: Individuos que utilizan Internet (% de la población)


In [4]:
# Lista de variables seleccionadas del Banco Mundial
SEvar = ['GB.XPD.RSDV.GD.ZS','IP.JRN.ARTC.SC', 'SP.POP.SCIE.RD.P6', 'SP.POP.TECH.RD.P6', 'NY.GDP.PCAP.CD', 'SE.XPD.TOTL.GD.ZS', 'SE.XPD.TOTL.GB.ZS', 'SE.PRM.ENRR', 'SE.SEC.ENRR','SE.TER.ENRR', 'ER.PTD.TOTL.ZS', 'AG.LND.FRST.K2', 'AG.SRF.TOTL.K2', 'GE.EST' , 'IT.NET.USER.ZS']

# Países participantes en GBIF
GBIF_countries = ['AND', 'AGO', 'ARG', 'ARM', 'AUS', 'BEL', 'BEN', 'BRA', 'BDI', 'KHM', 'CMR', 'CAN', 'CAF',
 'CHL', 'COL', 'CRI', 'HRV', 'DNK', 'ECU', 'EST', 'FIN', 'FRA', 'GEO', 'DEU', 'GTM', 'GIN',
 'ISL', 'IRL', 'LBR', 'LUX', 'MDG', 'MWI', 'MRT', 'MEX', 'MNG', 'NAM', 'NLD', 'NZL', 'NOR',
 'PAN', 'PER', 'POL', 'PRT', 'SLE', 'SVK', 'SVN', 'ZAF', 'SSD', 'ESP', 'SUR', 'SWE', 'CHE',
 'TJK', 'TGO', 'TON', 'GBR', 'USA', 'URY', 'UZB', 'ZWE']

# Carga de datos con la API del Banco Mundial
SEdata = wb.data.DataFrame(SEvar, GBIF_countries, time=range(2000, 2025), labels=True).reset_index()

# Pivotear y organizar la tabla en un solo paso
SEdata_pivot = (
    SEdata.drop(columns=['Series'])
    .melt(id_vars=["economy", "series", "Country"], var_name="Year", value_name="Value")
    .assign(Year=lambda df: df['Year'].str.replace('YR', '').astype(int))
    .pivot_table(index=["economy", "Country", "Year"], columns="series", values="Value")
    .reset_index()
    .rename_axis(None, axis=1)
)

SEdata_pivot.head()


,economy,Country,Year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,ER.PTD.TOTL.ZS,GB.XPD.RSDV.GD.ZS,GE.EST,IP.JRN.ARTC.SC,IT.NET.USER.ZS,NY.GDP.PCAP.CD,SE.PRM.ENRR,SE.SEC.ENRR,SE.TER.ENRR,SE.XPD.TOTL.GB.ZS,SE.XPD.TOTL.GD.ZS,SP.POP.SCIE.RD.P6,SP.POP.TECH.RD.P6
0,AGO,Angola,2000,777086.10,1246700.0,NaN,NaN,-1.075034,6.08,0.105046,563.733796,NaN,13.369070,NaN,5.25508,2.60753,NaN,NaN
1,AGO,Angola,2001,771535.49,1246700.0,NaN,NaN,NaN,9.58,0.136014,533.586202,NaN,15.001340,NaN,NaN,NaN,NaN,NaN
2,AGO,Angola,2002,765984.88,1246700.0,NaN,NaN,-1.132157,16.13,0.270377,882.147847,NaN,16.149599,0.78364,NaN,NaN,NaN,NaN
3,AGO,Angola,2003,760434.27,1246700.0,NaN,NaN,-1.124224,5.89,0.370682,992.698979,NaN,NaN,2.87621,NaN,NaN,NaN,NaN
4,AGO,Angola,2004,754883.66,1246700.0,NaN,NaN,-1.328872,9.73,0.464815,1266.210864,NaN,NaN,2.14597,NaN,NaN,NaN,NaN


In [5]:
#Esta fuente de variables fue descartada debido a la falta de datos completos para los países de interés y los años requeridos.

'''
# URL del archivo Excel en GitHub
url = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/ODIN_Data_2024.xlsx"

# OPTIMIZADO: Leer todas las hojas en un diccionario de DataFrames
all_sheets = pd.read_excel(url, sheet_name=None)

resultados = []
for year_str, df in all_sheets.items():
    year = int(year_str.replace(" Scores", "").strip())
    
    # Procesar y añadir a la lista
    processed_df = (
        df[df['Data category'] == 'All categories']
        [['Country code', 'Coverage subscore', 'Openness subscore', 'Overall score']]
        .assign(Year=year)
    )
    resultados.append(processed_df)

# Unir todo en un solo DataFrame
Odin_final = pd.concat(resultados, ignore_index=True)

# Asegurar que las columnas de puntajes sean numéricas
score_cols = ['Coverage subscore', 'Openness subscore', 'Overall score']
Odin_final[score_cols] = Odin_final[score_cols].apply(pd.to_numeric, errors='coerce')

print(Odin_final.head())
print("\nTipos de datos:")
print(Odin_final.dtypes)

# Asegurarse de que los tipos de datos de 'Year' coincidan
Odin_final['Year'] = Odin_final['Year'].astype(int)
SEdata_pivot['Year'] = SEdata_pivot['Year'].astype(int)

# Hacer el join usando Country code ↔ economy y Year ↔ Year
SE_odin = pd.merge(
    SEdata_pivot,
    Odin_final,
    left_on=['economy', 'Year'],
    right_on=['Country code', 'Year'],
    how='left'  
).drop(columns=['Country code'])

SE_odin.head()

'''

'\n# URL del archivo Excel en GitHub\nurl = "https://raw.githubusercontent.com/rortizgeo/Maestria_CD_Proyecto-Aplicado/main/ODIN_Data_2024.xlsx"\n\n# OPTIMIZADO: Leer todas las hojas en un diccionario de DataFrames\nall_sheets = pd.read_excel(url, sheet_name=None)\n\nresultados = []\nfor year_str, df in all_sheets.items():\n    year = int(year_str.replace(" Scores", "").strip())\n    \n    # Procesar y añadir a la lista\n    processed_df = (\n        df[df[\'Data category\'] == \'All categories\']\n        [[\'Country code\', \'Coverage subscore\', \'Openness subscore\', \'Overall score\']]\n        .assign(Year=year)\n    )\n    resultados.append(processed_df)\n\n# Unir todo en un solo DataFrame\nOdin_final = pd.concat(resultados, ignore_index=True)\n\n# Asegurar que las columnas de puntajes sean numéricas\nscore_cols = [\'Coverage subscore\', \'Openness subscore\', \'Overall score\']\nOdin_final[score_cols] = Odin_final[score_cols].apply(pd.to_numeric, errors=\'coerce\')\n\nprint(Odi

#### Llamado a los metadatos de paíes.

In [6]:
# Llamada API World Bank (metadatos actuales)
url = "https://api.worldbank.org/v2/country?format=json&per_page=300"
response = requests.get(url)
data = response.json()

countries = data[1]
countries_meta = pd.DataFrame([{
    "id": country["id"],
    "iso2Code": country["iso2Code"],
    "region": country["region"]["value"],
    "incomeLevel": country["incomeLevel"]["value"]
} for country in countries])

# Merge con el dataframe principal
BM_Data = pd.merge(SEdata_pivot, countries_meta, left_on="economy", right_on="id", how="left")

# Renombrar y eliminar columnas no necesarias
BM_Data = BM_Data.rename(columns={
    'Year': 'year',
    'Country': 'country',
    'iso2Code': 'countryCode'
}).drop(['economy', 'id'], axis=1)

# Extracción, limpieza y unión de datos de GBIF

Este segmento del script descarga, filtra y une automáticamente datos de GBIF, centrados en ocurrencias y especies reportadas en el país y por país publicador.
Finalmente une los cuatro DataFrames usando una unión externa (outer join) por year y countryCode para obtener como resultado una tabla integrada con los conteos por país y año, tanto desde el punto de vista del país donde ocurre la observación como del país que publica.

In [7]:
############################################
# Descargar los datos y generar subset por año y país
############################################

# Parámetros
BASE_URL = "https://analytics-files.gbif.org/global/csv/"
GBIF_FILES = [
    "occ_country.csv",
    "occ_publisherCountry.csv"
    # "spe_country.csv",
    # "spe_publisherCountry.csv"
]
DIAS_UMBRAL = 60  # días alrededor del 31 de diciembre a considerar

# Función para cargar archivos CSV desde URL
def load_csv(filename):
    url = BASE_URL + filename
    try:
        print(f"📥 Cargando {filename} desde: {url}")
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))
        print(f"✅ {filename} cargado con éxito.")
        return df
    except Exception as e:
        print(f"❌ Error al cargar {filename}: {e}")
        return pd.DataFrame()

# Función para ajustar el año según cercanía a diciembre 31
def ajustar_anio(snapshot, max_dias=DIAS_UMBRAL):
    d31_prev = datetime(snapshot.year - 1, 12, 31)
    d31_curr = datetime(snapshot.year, 12, 31)
    diff_prev = abs((snapshot - d31_prev).days)
    diff_curr = abs((snapshot - d31_curr).days)
    
    if diff_prev <= max_dias and snapshot < d31_curr:
        return snapshot.year - 1
    elif diff_curr <= max_dias:
        return snapshot.year
    else:
        return None

# Función para obtener solo la fecha más cercana a 31 dic por año
def extraer_snapshot_mas_cercano(df, dias_umbral=DIAS_UMBRAL):
    df = df.copy()
    df["snapshot"] = pd.to_datetime(df["snapshot"])
    df["year"] = df["snapshot"].apply(lambda x: ajustar_anio(x, dias_umbral))
    df = df.dropna(subset=["year"])
    
    df["dist_to_d31"] = df.apply(
        lambda row: abs((row["snapshot"] - datetime(int(row["year"]), 12, 31)).days),
        axis=1
    )
    
    df_sorted = df.sort_values(by=["year", "dist_to_d31"])
    fechas_cercanas = df_sorted.groupby("year")["snapshot"].first().reset_index()
    
    df_final = df.merge(fechas_cercanas, on=["year", "snapshot"], how="inner")
    return df_final.drop(columns=["dist_to_d31"])

# Cargar y procesar los archivos seleccionados
dataframes = {}
for filename in GBIF_FILES:
    key = filename.replace(".csv", "")
    df_raw = load_csv(filename)
    if not df_raw.empty:
        dataframes[key] = extraer_snapshot_mas_cercano(df_raw)

globals().update(dataframes)

# Renombrar columnas para merge
occ_publisherCountry.rename(columns={
    'publisherCountry': 'countryCode',
    'occurrenceCount': 'occurrenceCount_publisher'
}, inplace=True)

occ_country.rename(columns={'country': 'countryCode'}, inplace=True)

# Lista de datasets
datasets = [occ_country, occ_publisherCountry]

# Limpieza: eliminar snapshot y filtrar NaNs en countryCode
for i in range(len(datasets)):
    datasets[i] = datasets[i].drop(columns=["snapshot"], errors="ignore")
    datasets[i] = datasets[i].dropna(subset=["countryCode"])

# Merge usando year y countryCode como claves
from functools import reduce
df_merged = reduce(lambda left, right: pd.merge(left, right, on=["year", "countryCode"], how="outer"), datasets)

# Mostrar una vista previa
print(df_merged.head())

📥 Cargando occ_country.csv desde: https://analytics-files.gbif.org/global/csv/occ_country.csv
✅ occ_country.csv cargado con éxito.
📥 Cargando occ_publisherCountry.csv desde: https://analytics-files.gbif.org/global/csv/occ_publisherCountry.csv
✅ occ_publisherCountry.csv cargado con éxito.
  countryCode  occurrenceCount    year  occurrenceCount_publisher
0          AD          13343.0  2007.0                        NaN
1          AE           1192.0  2007.0                        NaN
2          AF          23395.0  2007.0                        NaN
3          AG           3286.0  2007.0                        NaN
4          AI           2372.0  2007.0                        NaN


### Unión de datos socieconómicos y de publicación de biodiversidad

In [8]:
Data = pd.merge(BM_Data, df_merged, on=["year", "countryCode"], how="left")
Data.head()

,country,year,AG.LND.FRST.K2,AG.SRF.TOTL.K2,ER.PTD.TOTL.ZS,GB.XPD.RSDV.GD.ZS,GE.EST,IP.JRN.ARTC.SC,IT.NET.USER.ZS,NY.GDP.PCAP.CD,...,SE.TER.ENRR,SE.XPD.TOTL.GB.ZS,SE.XPD.TOTL.GD.ZS,SP.POP.SCIE.RD.P6,SP.POP.TECH.RD.P6,countryCode,region,incomeLevel,occurrenceCount,occurrenceCount_publisher
0,Angola,2000,777086.10,1246700.0,NaN,NaN,-1.075034,6.08,0.105046,563.733796,...,NaN,5.25508,2.60753,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN
1,Angola,2001,771535.49,1246700.0,NaN,NaN,NaN,9.58,0.136014,533.586202,...,NaN,NaN,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN
2,Angola,2002,765984.88,1246700.0,NaN,NaN,-1.132157,16.13,0.270377,882.147847,...,0.78364,NaN,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN
3,Angola,2003,760434.27,1246700.0,NaN,NaN,-1.124224,5.89,0.370682,992.698979,...,2.87621,NaN,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN
4,Angola,2004,754883.66,1246700.0,NaN,NaN,-1.328872,9.73,0.464815,1266.210864,...,2.14597,NaN,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN


#### Membresías de 

Debido a que algunos datos vacíos en la variable objetivo corresponden al año en el que el país aún no era miembro de GBIF, se agrega dicha información a los datos con el fin de filtrarlos y eliminar filas que no son necesarias y así reducir el número de datos faltantes. Esta información se obtiene a partir de los datos de la red de GBIF https://www.gbif.org/the-gbif-network

In [9]:
gbif_membership_year = {
    "Andorra": 2010, "Angola": 2019, "Argentina": 2002, "Armenia": 2020, "Australia": 2001, 
    "Belgium": 2001, "Benin": 2004, "Brazil": 2012, "Burundi": 2024, "Cambodia": 2020, 
    "Cameroon": 2018, "Canada": 2001, "Central African Republic": 2011, "Chile": 2009, 
    "Colombia": 2003, "Costa Rica": 2001, "Croatia": 2022, "Denmark": 2001, 
    "Dominican Republic": 2025, "Ecuador": 2016, "Estonia": 2003, "Finland": 2001, 
    "France": 2001, "Georgia": 2023, "Germany": 2001, "Guatemala": 2021, "Guinea": 2005, 
    "Iceland": 2001, "Ireland": 2008, "Korea, Republic of": 2001, "Liberia": 2018, 
    "Luxembourg": 2008, "Madagascar": 2003, "Malawi": 2015, "Mauritania": 2009, 
    "Mexico": 2001, "Mongolia": 2025, "Namibia": 2021, "Netherlands": 2001, 
    "New Zealand": 2001, "Nigeria": 2016, "Norway": 2004, "Panama": 2024, "Peru": 2002, 
    "Poland": 2001, "Portugal": 2001, "Sierra Leone": 2022, "Slovakia": 2001, 
    "Slovenia": 2001, "South Africa": 2003, "South Sudan": 2018, "Spain": 2001, 
    "Sweden": 2001, "Switzerland": 2001, "Tajikistan": 2021, "Tanzania": 2002, 
    "Timor-Leste": 2024, "Togo": 2009, "Tonga": 2019, "United Kingdom": 2001, 
    "United States": 2001, "Uruguay": 2009, "Uzbekistan": 2020, "Zimbabwe": 2018
}

#  Convertir diccionario a DataFrame para el merge
gbif_membership_df = pd.DataFrame(
    gbif_membership_year.items(),
    columns=['country', 'gbif_start_year']
)

# Unir con el DataFrame principal
Data = pd.merge(Data, gbif_membership_df, on='country', how='left')

# Crear columna con np.where 
Data['gbif_member'] = np.where(
    Data['year'] >= Data['gbif_start_year'],
    'Sí',
    'No'
)

# Limpiar columna auxiliar
Data = Data.drop(columns=['gbif_start_year'])

Data[['country', 'year', 'gbif_member']].head()


,country,year,gbif_member
0,Angola,2000,No
1,Angola,2001,No
2,Angola,2002,No
3,Angola,2003,No
4,Angola,2004,No


### Opend Data Policy
El siguiente código permite la incluisión de una variable asociada a Políticas de Datos abiertos, esta fuente fue consultada según los países miembros del Open Government Partnership (https://www.opengovpartnership.org/our-members/#national). Hay que tener en cuenta que puede haber algunos países que cuenten con dicha política pero para los cuáles no se tenga información disponible de una fuente oficial.

In [10]:
ogp_membership_raw = {
    'Argentina': {'member': True,  'year': 2012}, 'Armenia': {'member': True,  'year': 2011}, 'Australia': {'member': True,  'year': 2015}, 'Benin': {'member': True,  'year': 2014},
    'Brazil': {'member': True,  'year': 2011}, 'Canada': {'member': True,  'year': 2012}, 'Chile': {'member': True,  'year': 2011}, 'Colombia': {'member': True,  'year': 2011},
    'Costa Rica': {'member': True,  'year': 2012}, 'Croatia': {'member': True,  'year': 2011}, 'Ecuador': {'member': True,  'year': 2018}, 'Estonia': {'member': True,  'year': 2011},
    'Finland': {'member': True,  'year': 2013}, 'France': {'member': True,  'year': 2014}, 'Georgia': {'member': True,  'year': 2011}, 'Germany': {'member': True,  'year': 2016},
    'Guatemala': {'member': True,  'year': 2011}, 'Ireland': {'member': True,  'year': 2014}, 'Liberia': {'member': True,  'year': 2011}, 'Malawi': {'member': True,  'year': 2022},
    'Mexico': {'member': True,  'year': 2011}, 'Mongolia': {'member': True,  'year': 2013}, 'Netherlands': {'member': True,  'year': 2011}, 'New Zealand': {'member': True,  'year': 2014},
    'Norway': {'member': True,  'year': 2011}, 'Panama': {'member': True,  'year': 2012}, 'Peru': {'member': True,  'year': 2011}, 'Portugal': {'member': True,  'year': 2012},
    'Sierra Leone': {'member': True,  'year': 2013}, 'Slovakia': {'member': True,  'year': 2011}, 'South Africa': {'member': True,  'year': 2011}, 'Spain': {'member': True,  'year': 2011},
    'Sweden': {'member': True,  'year': 2011}, 'United Kingdom': {'member': True,  'year': 2011}, 'United States': {'member': True,  'year': 2011}, 'Uruguay': {'member': True,  'year': 2012}
}

# Pre-procesar el diccionario para facilitar el merge
ogp_data = [
    {'country': country, 'ogp_start_year': info['year']}
    for country, info in ogp_membership_raw.items() if info.get('member')
]
ogp_df = pd.DataFrame(ogp_data)

# Unir y crear la columna con np.where
Data = pd.merge(Data, ogp_df, on='country', how='left')
Data['ogp_membership'] = np.where(
    Data['year'] >= Data['ogp_start_year'],
    'Sí',
    'No'
)

# Limpiar columna auxiliar
Data = Data.drop(columns=['ogp_start_year'])

Data[['country', 'year', 'ogp_membership']].head()



,country,year,ogp_membership
0,Angola,2000,No
1,Angola,2001,No
2,Angola,2002,No
3,Angola,2003,No
4,Angola,2004,No


In [11]:
rename_dict = {
    'IP.JRN.ARTC.SC': 'art_cientificos', 'SP.POP.SCIE.RD.P6': 'investigadores_RD', 'GB.XPD.RSDV.GD.ZS': 'gasto_RD_pib',
    'SP.POP.TECH.RD.P6': 'tecnicos_RD', 'NY.GDP.PCAP.CD': 'pib_per_capita', 'SE.XPD.TOTL.GD.ZS': 'gasto_educacion_pib',
    'SE.PRM.ENRR': 'inscripcion_primaria', 'SE.SEC.ENRR': 'inscripcion_secundaria', 'SE.TER.ENRR': 'inscripcion_terciaria',
    'SE.XPD.TOTL.GB.ZS': 'gasto_educacion_gobierno', 'ER.PTD.TOTL.ZS': 'areas_protegidas', 'AG.LND.FRST.K2': 'area_selvatica_km2',
    'AG.SRF.TOTL.K2': 'superficie_total_km2', 'GE.EST': 'efectividad_gobierno', 'IT.NET.USER.ZS': 'uso_internet'
}

Data = Data.rename(columns=rename_dict)
Data.head()

,country,year,area_selvatica_km2,superficie_total_km2,areas_protegidas,gasto_RD_pib,efectividad_gobierno,art_cientificos,uso_internet,pib_per_capita,...,gasto_educacion_pib,investigadores_RD,tecnicos_RD,countryCode,region,incomeLevel,occurrenceCount,occurrenceCount_publisher,gbif_member,ogp_membership
0,Angola,2000,777086.10,1246700.0,NaN,NaN,-1.075034,6.08,0.105046,563.733796,...,2.60753,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN,No,No
1,Angola,2001,771535.49,1246700.0,NaN,NaN,NaN,9.58,0.136014,533.586202,...,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN,No,No
2,Angola,2002,765984.88,1246700.0,NaN,NaN,-1.132157,16.13,0.270377,882.147847,...,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN,No,No
3,Angola,2003,760434.27,1246700.0,NaN,NaN,-1.124224,5.89,0.370682,992.698979,...,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN,No,No
4,Angola,2004,754883.66,1246700.0,NaN,NaN,-1.328872,9.73,0.464815,1266.210864,...,NaN,NaN,NaN,AO,Sub-Saharan Africa,Lower middle income,NaN,NaN,No,No


In [12]:
import os
# Ruta relativa a la carpeta de tu repositorio Git
repo_path = "/Users/ricardoortiz/Desktop/Maestria_CD/ProyectoAplicado/RepositorioGit"
file_path = os.path.join(repo_path, "Data.csv")
Data.to_csv(file_path, index=False, encoding="utf-8")